In [1]:
import pandas as pd
import plotly.express as px

In [2]:
clients = pd.read_csv("data/clients.csv").drop(columns=['Unnamed: 0'])
transactions = pd.read_csv("data/transactions.csv").drop(columns=['Unnamed: 0'])

transactions = transactions.rename(columns={"ACCOUNT_ID":"POC"})
transactions["INVOICE_DATE"] = pd.to_datetime(transactions["INVOICE_DATE"], format="%Y%m%d")

In [3]:
print(f'Orders amount: {transactions["ORDER_ID"].nunique()}')
print(f'Products amount: {transactions["SKU_ID"].nunique()}')

Orders amount: 45547
Products amount: 530


In [4]:
pocs1 = set(clients["POC"])
pocs2 = set(transactions["POC"])
pocsboth = pocs1 & pocs2

print(f"Clients in Client's DB: {len(pocs1)}, Clients in Transaction's DB {len(pocs2)}, Clients in both DBs {len(pocsboth)}")

clients = clients[clients["POC"].isin(pocsboth)].copy()
transactions = transactions[transactions["POC"].isin(pocsboth)].copy()

Clients in Client's DB: 4400, Clients in Transaction's DB 4535, Clients in both DBs 4379


Let's make [POC, INVOICE_DATE, SKU_ID] a primary key combination

In [5]:
transactions = (transactions[["POC","INVOICE_DATE"]].drop_duplicates()
    .merge((transactions
            .groupby(["POC","INVOICE_DATE","SKU_ID"])
            ["ITEMS_PHYS_CASES"].sum()
            .reset_index()
        ),
        on=["POC","INVOICE_DATE"],
        how="left"
    )
    .sort_values(["POC","INVOICE_DATE","SKU_ID"])
)


transactions = (transactions
    .merge((transactions
            [["POC","INVOICE_DATE"]]
            .drop_duplicates()
            .reset_index(drop=True)
            .reset_index()
            .rename(columns={"index":"ORDER_ID"})
        ),
        on=["POC","INVOICE_DATE"],
        how="left"
    )
    [["POC","ORDER_ID","INVOICE_DATE","SKU_ID","ITEMS_PHYS_CASES"]]
    .sort_values(["POC","ORDER_ID","INVOICE_DATE","SKU_ID"])
)

Let's visualize the amount of orders per product

In [6]:
orders_per_product = transactions["SKU_ID"].value_counts().rename("orders_per_product")
mean_orders_per_product = orders_per_product.mean()

plot_threshold = 1000
title = f"Histogram of orders per product (mean: {int(mean_orders_per_product)}) [plot threshold < {plot_threshold}]"
px.histogram(orders_per_product[orders_per_product < plot_threshold], nbins=100, title=title).show()

Let's eliminate products that been ordered less than N=25 times

In [7]:
N = 50

invalid_products = orders_per_product[orders_per_product < N].index.values

print(f"Dropping {len(invalid_products)} products due to less than {N} orders")

transactions = transactions[~transactions["SKU_ID"].isin(invalid_products)].copy()

print(f"Remaining products {transactions['SKU_ID'].nunique()}")

Dropping 200 products due to less than 50 orders
Remaining products 330


Let's eliminate products that been ordered at most N=3 by any client

In [8]:
N=3

orders_perClientProduct = transactions[["POC","SKU_ID"]].value_counts().reset_index()
maxOrders_perClientProduct = orders_perClientProduct.groupby("SKU_ID")["count"].max().sort_values()

invalid_SKUIDs = maxOrders_perClientProduct[maxOrders_perClientProduct <= N].index.values

print(f"Dropping {len(invalid_SKUIDs)} products due to less than 3 max orders by any client")

transactions = transactions[~transactions["SKU_ID"].isin(invalid_SKUIDs)].copy()

print(f"Remaining products {transactions['SKU_ID'].nunique()}")

Dropping 53 products due to less than 3 max orders by any client
Remaining products 277


Stage results

In [9]:
clients.to_pickle("data/parsed/clients.pkl")
transactions.to_pickle("data/parsed/transactions.pkl")